In [1]:
from IPython.display import display, HTML
import pandas as pd
import time
import re
import os
import sys
import psycopg2
import ipywidgets as widgets
from bokeh.io import output_file, show
from bokeh.models import CheckboxGroup
from ipywidgets import interact, interactive, fixed, Layout, HBox, VBox

# Project functions and classes
if './modules/' not in sys.path: sys.path.append('./modules')
from widgets import *
from selection import *
from build_statements import *
from query import *
from plot import *   
from outliers import *

def query_plot(b):
    start = time.time()
    default = ('ignore','   ')
    dd_default = ('ignore',)
    
    with output:
        # Prevalence value to tuple
        b = [1]
        b[0] = prevalence_indications_dd.value
        prevalence_indications_dd_tuple = tuple(b)
        b = [1]
        b[0] = deprivation_dd.value
        deprivation_indications_dd_tuple = tuple(b)
        
        features_values={'prescribing':[bnf_code_dd.value,'bnf_code',prescribing_measures_dd.value,\
                                        month_dd.value,year_dd.value],\
                'prevalence':[prevalence_indications_dd_tuple,'indication',\
                              prevalence_measures_dd.value,'',year_dd.value],\
                'gender':[gender_dd.value,'sex',gender_age_measures_dd.value,\
                          month_dd.value,year_dd.value],\
                'age_groups':[age_groups_dd.value,'age_group',gender_age_measures_dd.value,\
                              month_dd.value,year_dd.value],\
                'deprivation':[deprivation_indications_dd_tuple,'deprivation_index',\
                               deprivation_measures_dd.value,'',year_dd.value]}
        
        headers = ['value','value_header','measure_header','month','year']
        fvdf = pd.DataFrame.from_dict(features_values, orient='index',columns=headers)
        fvdf = fvdf[~fvdf['value'].isin([default,dd_default])]
        fvdf=fvdf.fillna('not_selected')
        features = list (fvdf.index)
        tables = get_tables(fvdf,features)
        fvdf['table'] = tables
        
        # Validate the selection so that all fields are provided correctly  
        error, comment = validate (fvdf,features)
        if error:
            display(HTML('<h3 style="color:red">{comment}</h1>'.format (comment = comment)))
        
        else:
            # Use the query to build a title for the plot
            features_titles=get_title(fvdf)

            # Build query statements for the features selected
            statements = {}

            # If both age and gender were selected
            if ({'age_groups','gender'}) <= set(features):
                built_statement = genderANDage_statement(fvdf).build()
                statements['gender_age_groups'] = built_statement
                features = [feature for feature in features if feature not in ['age_groups','gender']]

            for feature in features:
                st = statement (feature,fvdf)
                built_statement = '{sel}\n{whe}\n{grp}'.format (sel=st.select (feature),\
                whe=st.where(feature),grp=st.groupby(feature))
                statements[feature] = built_statement

            # Locations data
            location_statement = "SELECT practice,practice_code,ccg,region,sub_region,\
                                  longitudemerc,latitudemerc\
                                  FROM practices_locations"
            statements['location']=location_statement

            # Use the statement to query the relevant tables
            queries_df=query(statements)

            # Check outliers in value columns
            outliers_choice_selection = outliers_choice.value
            if outliers_choice_selection == 'Remove':
                geo_columns=['practice_code', 'practice', 'ccg','region','sub_region', 'longitudemerc','latitudemerc']
                value_headers=[h for h in queries_df.columns if h not in geo_columns]
                outliers_indices_collect=[]
                for header in value_headers:
                    outliers_indices = detect_indices(queries_df,header)
                    outliers_indices_collect+=outliers_indices
                queries_df=queries_df[~queries_df.index.isin(outliers_indices_collect)]
            
            # Download the joined data for the user
            download_choice_selection = download_choice.value
            if download_choice == 'CSV':
                queries_df.to_csv('queries_df.csv',index=False)
            elif download_choice == 'Excel':
                queries_df.to_csv('queries_df.xlsx',index=False)
            
            # Execution time
            print ('query execution time = {t} seconds'.format (t=round (time.time()-start))) 
            
            # Generate plot
            output_choice_selection = output_choice.value
            map_plot(queries_df,features_titles, output_choice_selection)
        
        
# Dashboard
output = widgets.Output()
button.on_click(query_plot)
display(HTML('<h1 style="color:magenta">Geo Health</h1>\
              <p style="color:blue">Select one or two of the following features. You can select multiple values for each:</p>\
                 <ul>\
                  <li style="color:magenta">Prescribing: Monthly, 2018-19</li>\
                  <li style="color:magenta">Prevalence: Annual, 2014-19</li>\
                  <li style="color:magenta">Age and Gender: Monthly, 2018-19</li>\
                  <li style="color:magenta">Deprivation: 2015, 2019</li>\
                </ul>'))

VBox(children=[tabs,button_date_box,output])

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')